## Algorithmic Trading Framework
Data is from testnet, and therefore not accurate.

Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [76]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as sco
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
# pd.reset_option('display.float_format')

In [77]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
pd.to_datetime(client.time()['serverTime'], unit='ms')

Timestamp('2022-05-10 08:31:10.941000')

If your time is off you need to synchronize with timeserver

In [78]:
balance = pd.json_normalize(client.account()['balances'])
balance

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [79]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]

In [80]:
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols)
assets

BTCUSDT                                                   \
       Open time            Open             High             Low   
0  1651622400000  38127.16000000   67952.01000000   9000.00000000   
1  1651708800000  39690.01000000   48256.09000000  12000.00000000   
2  1651795200000  36552.97000000   90000.00000000   9500.00000000   
3  1651881600000  35851.70000000  179685.00000000   2000.00000000   
4  1651968000000  35472.40000000  172120.00000000   6988.00000000   
5  1652054400000  34036.87000000   44600.00000000   6392.00000000   
6  1652140800000  30089.10000000   32645.94000000  24600.00000000   

                                                                     \
            Close         Volume     Close time  Quote asset volume   
0  39695.80000000  3289.47888900  1651708799999  128626868.06394152   
1  36551.04000000  3747.85911700  1651795199999  142179013.42520536   
2  35844.54000000  5133.87878000  1651881599999  185618082.67083564   
3  35472.39000000  2648.74837000  1651967999999   94693433.70235498   
4  34035.00000000  5928.73934500  1652054399999  204738551.12075512   
5  30092.43000000  7687.56723900  1652140799999  249999769.04934389   
6  31955.57000000  3075.50862500  1652227199999   96094163.49760137   

                                                ...     XRPUSDT              \
  Number of trades Taker buy base asset volume  ...        High         Low   
0           109403               2070.21409400  ...  0.64990000  0.60830000   
1           136426               2305.80703300  ...  0.65710000  0.58040000   
2           195320               2791.12783400  ...  0.61630000  0.58130000   
3           110202               1439.33242400  ...  0.60340000  0.56940000   
4           216686               3380.01233600  ...  0.58280000  0.55810000   
5           259088               4741.36570900  ...  0.57870000  0.47290000   
6            99494               1981.96682100  ...  0.49490000  0.48270000   

                                                                    \
        Close             Volume     Close time Quote asset volume   
0  0.64610000   7188003.90000000  1651708799999   4497422.00637000   
1  0.59830000  11682556.70000000  1651795199999   7147202.83414000   
2  0.58130000  12824794.10000000  1651881599999   7706389.17155000   
3  0.58180000   4378006.60000000  1651967999999   2579982.36840000   
4  0.56620000   9985261.30000000  1652054399999   5708676.23890000   
5  0.48650000  27793876.20000000  1652140799999  14549295.78418000   
6  0.48640000    382119.60000000  1652227199999    186851.99706000   

                                                                             \
  Number of trades Taker buy base asset volume Taker buy quote asset volume   
0             8638            7020708.90000000             4393175.81487000   
1            13976           11566933.90000000             7074742.84037000   
2            15203           12610238.80000000             7578080.05335000   
3             5475            4097615.10000000             2413970.94234000   
4            10831            9690239.70000000             5539919.86459000   
5            28997           27405138.00000000            14344952.93562000   
6              636             330533.40000000              161839.68548000   

          
  Ignore  
0      0  
1      0  
2      0  
3      0  
4      0  
5      0  
6      0  

[7 rows x 72 columns]

Swapping levels for easier selection 

In [81]:
assets = assets.swaplevel(axis=1)

In [124]:
assets = assets.set_index(pd.to_datetime(assets['Close time', 'BTCUSDT'], unit='ms').dt.strftime('%Y-%m-%d')) # Set close time as index, needs improvement
assets.index.name = 'Close time'

## Daily Returns

In [125]:
assets_close = assets["Close"].copy().astype(float) # Daily close prices
assets_close

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
Close time,,,,,,
2022-05-04,39695.8000,2940.6700,402.6000,106.4000,0.0863,0.6461
2022-05-05,36551.0400,2747.9300,378.5000,96.9000,0.0745,0.5983
2022-05-06,35844.5400,2655.3300,157.0700,95.6000,0.0817,0.5813
2022-05-07,35472.3900,2637.3300,365.5000,94.5000,0.0835,0.5818
2022-05-08,34035.0000,2519.5100,356.0000,94.0000,0.0880,0.5662
2022-05-09,30092.4300,2227.2500,296.4000,76.0000,0.0727,0.4865
2022-05-10,31955.5700,2843.5400,326.7000,76.9000,0.0720,0.4864


In [126]:
returns = assets_close.pct_change().dropna()

In [127]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
Close time,,,,,,
2022-05-05,-0.0792,-0.0655,-0.0599,-0.0893,-0.1363,-0.0740
2022-05-06,-0.0193,-0.0337,-0.5850,-0.0134,0.0957,-0.0284
2022-05-07,-0.0104,-0.0068,1.3270,-0.0115,0.0223,0.0009
2022-05-08,-0.0405,-0.0447,-0.0260,-0.0053,0.0534,-0.0268
2022-05-09,-0.1158,-0.1160,-0.1674,-0.1915,-0.1731,-0.1408


In [128]:
returns.mean(axis = 0)

BTCUSDT   -0.0339
ETHUSDT    0.0017
BNBUSDT    0.0985
LTCUSDT   -0.0499
TRXUSDT   -0.0248
XRPUSDT   -0.0449
dtype: float64

## Annualised Risk, Return & Sharpe

In [87]:
def annualised_risk_return(returns):
    stats = returns.agg(['mean', 'std']).T
    stats.columns = ['Return', 'Risk']
    stats.Return = stats.Return*365.25 # Crypto exchanges trade every day, including leap days.
    stats.loc[stats.Return < -1, 'Return'] = -1 # Set losses > 100% to -100%
    stats.Risk = stats.Risk * np.sqrt(365.25)
    return stats 

In [88]:
stats = annualised_risk_return(returns) # Risk may not be accurate for mean daily losses > 0.274% ()

In [89]:
riskfree_return = 0.031 # 5 Year Treasury Rate, but testnet resets every month

In [90]:
stats['Sharpe'] = stats['Return'].sub(riskfree_return)/stats['Risk']
stats

,Return,Risk,Sharpe
BTCUSDT,-1.000000,1.170571,-0.880767
ETHUSDT,0.609477,2.668470,0.216782
BNBUSDT,35.972647,12.349303,2.910419
LTCUSDT,-1.000000,1.485445,-0.694068
TRXUSDT,-1.000000,2.048599,-0.503271
XRPUSDT,-1.000000,1.037176,-0.994045


## Optimal Sharpe Ratio Portfolio

In [91]:
asset_qty = len(assets_close.columns)

In [92]:
def portfolio_return(weights):
    return returns.dot(weights.T).mean() * 365.25

In [93]:
def portfolio_risk(weights):
    return returns.dot(weights.T).std() * 365.25

In [94]:
def minimized_sharpe(weights):
    return (riskfree_return - portfolio_return(weights))/portfolio_risk(weights)

In [95]:
equal_weights = np.full(asset_qty, 1/asset_qty)

In [96]:
constraint = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1})

In [97]:
bounds = tuple((0,1) for x in range(asset_qty))

In [120]:
# Sequential Least Squares Programming
optimum = sco.minimize(minimized_sharpe, equal_weights, method='SLSQP', bounds=bounds, constraints=constraint)

In [114]:
np.set_printoptions(suppress = True)
optimum # No surprise with that 157.07$ BNB outlier

     fun: -0.1522861582901173
     jac: array([ 0.0560376 ,  0.00197842, -0.00013135,  0.08159326,  0.03998628,
        0.0747871 ])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0., 0., 1., 0., 0., 0.])

In [115]:
pd.options.display.float_format = '{:.4f}'.format
optimal_weights = optimum['x']
optimal_weights = pd.Series(index = assets_close.columns, data = optimal_weights)
optimal_weights

BTCUSDT   0.0000
ETHUSDT   0.0000
BNBUSDT   1.0000
LTCUSDT   0.0000
TRXUSDT   0.0000
XRPUSDT   0.0000
dtype: float64

In [116]:
portfolio_return(optimal_weights)

35.972647413081965

In [117]:
portfolio_risk(optimal_weights)

236.01388213241455

In [118]:
-minimized_sharpe(optimal_weights)

0.1522861582901173